#### Names of people in the group

Please write the names of the people in your group in the next cell.

Name of person A: Dominik Ucher

Name of person B

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-cab8db48-3496-434e-826c-9d6dfdd67033/bin/python -m pip install --upgrade pip' command.


In [0]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users.parquet")
comments_df = load_df("/user/hive/warehouse/comments.parquet")
posts_df = load_df("/user/hive/warehouse/posts.parquet")

#### Subtask 1: implenenting two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [0]:
def run_query(query: str, df: DataFrame) -> DataFrame:
    df.createOrReplaceTempView("df")
    result = spark.sql(query)

    first_row = result.first()
    first_column_value = first_row[0]
    return first_column_value


def run_query2(query: str, df1: Any, df2: Any) -> Any:
    df1.createOrReplaceTempView("df1")
    df2.createOrReplaceTempView("df2")
    result = spark.sql(query)
    
    first_row = result.first()
    first_column_value = first_row[0] 
    return first_column_value


In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [0]:
q1 = "SELECT * FROM df ORDER BY CreationDate DESC limit 1"

q2 = "SELECT COUNT(DISTINCT Id) FROM df"

q3 = "SELECT OwnerUserId FROM df WHERE PostTypeId=2 GROUP BY OwnerUserId ORDER BY COUNT(*) DESC limit 1"

q4 = "SELECT COUNT(*) FROM df WHERE PostTypeId=1"

q5 = "SELECT df1.DisplayName FROM df1 JOIN df2 ON df2.UserId=df1.Id GROUP BY df1.DisplayName ORDER BY COUNT(*) DESC LIMIT 1"


#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 8.794s

OK
Out[16]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

#### 1.
All three used for data extraction. RDD is the most fundamental data structure in Spark. RDD offers two types of operations that are transformations and actions. The advantage of using RDD if performance, consistency and fault tolerance. That it stores data in memory as well as parrallel processing. The contents of an RDD cannot be modified, providing stable data. And RDDs are resilient and compute missing/damaged parts

Spark DataFrame is a set of objects organized into a table (Columns and Rows). A Dataset is similar to a DataFrame but the data is easier to work with from a user standpoint

#### 2
You usually use RDDs when the data is unstructured, such as media or text streams. When transformations are low, making data manipulation fast and straight forward. And when schema is not important, since RDDs do not use schemas. You use Datasets when the data is structered, because dataframes infer shcema's on structed data. When tranformations are high-level and a lot to process. And when a high degree of safety is necessary, because DataFrame and DataSets are type-safety and takes full advantage of the speed of development 

#### 3
The main benefit of using Datasets instead of DataFrames is Safety. Datasets allow you to work with data in type-safe manner, while you cannot do the same in DataFrames. datasets are also better to optimize opportunities compared to DataFrames. So overall Datasets are more safer and more efficient way to work with structed and un-structed data than DataFrames